# Fase 3: Extracción de Detalles de Actores con Selenium

Extraer información detallada de los 10 principales actores de cada una de las películas extraidas en la fase 1 utilizando Selenium (desde la página e IMDB). Debereis solo tener en cuenta los 10 primeros actores de cada película o corto (en caso de que los haya)La información que deberás extraer es:

Nombre.

Año nacimiento.

¿Por qué es conocido?

¿Qué hace?

Premios.

NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. 



In [1]:
import requests
import re

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

from time import sleep

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [2]:

def obtener_detalles_actores(lista_actores,lista_detalles_actores): #obtenemos los detalles de los actores

    # Crear un diccionario llamado "datos_actor" para almacenar la información de cada uno de los actores. Un diccionario por cada actor.
    datos_actor = {}

    url_actores=(f"https://www.imdb.com/")

    # Inicializa el navegador Chrome
    driver = webdriver.Chrome()

    # Navega a la página web "https://www.imdb.com/"
    driver.get(url_actores)

    driver.maximize_window()

    sleep(2)

    # Aceptar las cookies en la página web

    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

    sleep(3)

    #Vamos a recorrer la lista de actores

    for actor in lista_actores:

            # Ingresar el actor de la lista de actores y coger los datos en el campo de búsqueda y presionar Enter
            
            driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)

            sleep(5)

            # Agrego el valor actor a la clave nombre_actor
            
            datos_actor["nombre_actor"] = actor
            
            # pinchamos en el resultado de la busqueda que nos lleva a la info del actor

            driver.execute_script("window.scrollTo(0,400)")
            
            sleep (2)
            
            driver.find_element('xpath','//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[1]/div[2]/div/a').click()
            
            sleep (3)

            # buscamos que_hace y agregamos el valor a la clave que_hace. Utilizamos try except para manejar los errores y evitar que el codigo se pare.

            try:
                resultado_que_hace = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text
                lista_que_hace = resultado_que_hace.split("\n")
                datos_actor["que_hace"] = lista_que_hace    
            
            except:                
                print("Error al encontrar la seccion 'que hace'")
                datos_actor["que_hace"] = "None"
            
            sleep(2) 

            # buscams el ano de nacimiento del actor y agregamos el valor obtenido a la clave anno_nacimiento

            try:            
                driver.execute_script("window.scrollTo(0,500)")
                anno_nacimiento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text
                resultado_anno_nacimiento = re.search(r'\b\d{4}\b', anno_nacimiento).group()
                datos_actor["anno_nacimiento"] = resultado_anno_nacimiento    
            except:
                print("Error al extrear el año de nacimiento")
                datos_actor['anno_nacimiento'] = "0"

            # buscamos los premios y agregams el valor a la clave premios

            sleep(3) 
            i = 500
            while i<2000:
                try:
                    driver.execute_script(f"window.scrollTo(0,{i})")
                    sleep(3)
                    
                    try:
                        texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul').text
                        break
                    except:
                        texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul').text
                        break
                except:
                    i+=500
            
            try:
                cantidad_premios = re.search(r'(\d+)\s+premios?',texto_premios, re.IGNORECASE).group(1)
                datos_actor['premios'] = cantidad_premios
        
            except:
                print("Error al extrear los premios")
                datos_actor['premios'] = "0"

            driver.execute_script(f"window.scrollTo({i},0)") 
            
            sleep(4)

            # buscamos las peliculas por las cuales el actor es conocido, las añadimos a una lista y agregamos la lista obtenida a la clave peliculas.

            listas_pelis_conocido_por = []
            for x in range(1,5):
                    
                try:
                    driver.execute_script("window.scrollTo(0,2000)")
                    sleep(3)
                    resultado_conocido_por = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-af50cbe8-0.fqoRPX > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({x}) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top').text
                    listas_pelis_conocido_por.append(resultado_conocido_por)
                except:                    
                    print("Error al encontrar las peliculas")                    
                    listas_pelis_conocido_por.append("None")

            sleep(2)

            datos_actor["peliculas"] = listas_pelis_conocido_por 
            
            #driver.back()

            # creamos la tupla para almacenar los datos obtenidos

            try:
                tupla_actor = (datos_actor['nombre_actor'], int(datos_actor['anno_nacimiento']), datos_actor["peliculas"], datos_actor["que_hace"], int(datos_actor['premios']))
                print(type(tupla_actor))
            except:
                print("Error al crear la tupla. Creamos tupla con null")
                tupla_actor = (datos_actor['nombre_actor'], 0, ["None"], ["None"], 0)

            lista_detalles_actores.append(tupla_actor)
                        
            datos_actor = {}
        
    print("----------------")
    
    #print(lista_detalles_actores)

    return(lista_detalles_actores)

In [4]:

def obtener_actores_from_peliculas(lista_id_peliculas, lista_actores): # obtenemos los 10 primeros actores de cada pelicula extraida en la fase 1

        url_actores=(f"https://www.imdb.com/")

        # Inicializa el navegador Chrome
        driver = webdriver.Chrome()

        # Navega a la página web "https://www.imdb.com/"
        driver.get(url_actores)

        driver.maximize_window()

        sleep(2)

        # Aceptar las cookies en la página web

        driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

        sleep(2)

        # Recorro la lista de id_peliculas 

        for pelicula in lista_id_peliculas:
                print(pelicula)
                
                try:
                        driver.find_element("css selector", "#suggestion-search").send_keys(pelicula, Keys.ENTER)
                except:
                        print("Pelicula no encontrada")

                driver.execute_script("window.scrollTo(0,500)")
                
                sleep(2)

                # Encontramos los 10 primeros actores

                for i in range(1,11):

                        try:
                                actor = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i})').text.split("\n")[0]                               
                                if actor not in lista_actores:
                                        lista_actores.append(actor)
                                        
                        except:
                                print(f"Error al encontrar los 10 primeros actores de la pelicula {pelicula}")
                sleep(2)                               
                driver.back()
                sleep(2)
                
        return lista_actores

In [15]:
import pandas as pd

# Pasamos el CSV obtenido en la fase 1 para ser leido

csv_url = "https://raw.githubusercontent.com/fernandaMarti/Proyecto-da-promo-H-modulo-2-team1-cine_freaks/main/Fase1.csv"

#df= pd.read_csv(csv_url,names=['tipo_pelicula','titulo_pelicula','anno_estreno','mes_estreno','id_pelicula','genero_pelicula'])

df= pd.read_csv(csv_url)

df.columns=['tipo_pelicula','titulo_pelicula','anno_estreno','mes_estreno','id_pelicula','genero_pelicula']

# Creamos una variable global para no perder los datos obtenidos

lista_actores = []

# Del DataFrame solo queremos acceder a la columna especifica "id_pelicula" convertida en lista

lista_id_peliculas = list(df['id_pelicula'])

lista_id_peliculas_1=lista_id_peliculas[0:500]

print(lista_id_peliculas)

# llamamos a la funcion

resultados_lista_actores = obtener_actores_from_peliculas(lista_id_peliculas_1, lista_actores) # obtener lista de actores usando una lista de id peliculas de un CSV


['tt1929361', 'tt29009897', 'tt3407398', 'tt8042420', 'tt4209582', 'tt4192098', 'tt1955042', 'tt9345952', 'tt7107042', 'tt3122402', 'tt5744122', 'tt14404560', 'tt10229482', 'tt14949656', 'tt15714220', 'tt4799362', 'tt3901262', 'tt9314692', 'tt5861858', 'tt2395146', 'tt8545442', 'tt3123212', 'tt2631168', 'tt11078340', 'tt13263988', 'tt1956409', 'tt27458934', 'tt6674088', 'tt3975556', 'tt7754786', 'tt29573299', 'tt27249903', 'tt2607520', 'tt3058660', 'tt11306766', 'tt3492290', 'tt3031866', 'tt8269236', 'tt5715042', 'tt3543308', 'tt5860550', 'tt3170428', 'tt29274814', 'tt6029778', 'tt4917660', 'tt3439212', 'tt4555154', 'tt22096758', 'tt2626926', 'tt7352942', 'tt9604058', 'tt3415998', 'tt31728011', 'tt32247927', 'tt5240372', 'tt11394766', 'tt2390528', 'tt4918006', 'tt7573274', 'tt2904472', 'tt8561086', 'tt6114010', 'tt6864586', 'tt6250952', 'tt3650008', 'tt4942210', 'tt4175660', 'tt31778427', 'tt3145882', 'tt1956417', 'tt8018696', 'tt11023700', 'tt4185566', 'tt2160137', 'tt1776160', 'tt205

In [ ]:
len(lista_id_peliculas_1)

In [13]:
lista_id_peliculas_1[0]

'tt1929361'

In [14]:
lista_id_peliculas[499]

'tt18266556'

In [17]:
import csv
with open('actores_def.csv', 'w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerow(lista_actores)

In [8]:
csv.reader("actores_def.csv")

In [23]:
import csv
lista_actores_def = []
with open('actores_def.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        print(','.join(row))
        lista_actores_def.append(row)
print(lista_actores_def)


Scott Arnold,Nathan Boreiko,Tj Carr,Jordy Dickens,Bill Fischer,Ian Hagan,Robert Harris,Matt Holtry,Christopher Houldsworth,Lela King,Ren Holly Liu,Alexandra Tobias,Sam Morgan,Joshua Polit,Patrick Sanderson,Josh Pafchek,Anishka Nawagamuwa,Maggie Mcgee,Victoria Tucha,Shayan Siren,Corey Camperchioli,Paul Castro Jr.,Donavon Dietz,Elizabeth Evans,Justin Rodriguez,Lilli Stein,Jared Bajoras,Patrick Fetzko,Samantha Strelitz,Sé Marie Volk,Dezz Peterson,Imani Jackson,Kasie Spencer,Tamicka Scruggs,Donald Don P. Peterson,Eric Graves,Theresa Burrage,Andrea Heard,Dachelle Bufford,LaTonia Young,Austin Butler,Gester Veas,Jordan Pryor,Paolo Fronticelli,Paola Giancaterino,Édouard Court,Félix Kysyl,Ludovic Berthillot,Mauricette Gourdon,Christophe Tek,Desiree Bailey,Sakina Flint,Terry Hodge,Anthony Johnson,Marcus Zollicoffer,Imran Arooj,Firdaus Jamal,Syed Faris Pervez,Saba Qamar,Pavitra Jain,Ojas Ujjwal Narula,Prahlad Rajput,Hrituraj Shrivastav,Liz Lafontant,Tarik Lewis,Kevin Julio,Indah Permatasari,Mathi

In [24]:
df_lista_actores_1 = pd.DataFrame(lista_actores_def[0])

In [25]:
df_lista_actores_1

,0
0,Scott Arnold
1,Nathan Boreiko
2,Tj Carr
3,Jordy Dickens
4,Bill Fischer
...,...
2488,Katerina Alexová
2489,Krystof Pipek
2490,Zuzana Lisková
2491,Kristýna Líbalová


In [27]:
df_lista_actores_1.to_csv("lista_actores_1.csv", index=False)

In [28]:
pd.read_csv('lista_actores_1.csv')

,0
0,Scott Arnold
1,Nathan Boreiko
2,Tj Carr
3,Jordy Dickens
4,Bill Fischer
...,...
2488,Katerina Alexová
2489,Krystof Pipek
2490,Zuzana Lisková
2491,Kristýna Líbalová


In [93]:
len(lista_intermedia_actores_2)

271

In [94]:
lista_intermedia_actores_2[0]

'Macy Minear'

In [95]:
lista_intermedia_actores_2[270]

'Makena Wall'

In [5]:
# creamos una variable global para no perder los datos obtenidos

lista_detalles_actores=[]

# llamamos a la funcion

resultado_detalles_actores = obtener_detalles_actores(lista_intermedia_actores_2,lista_detalles_actores)

Error al extrear el año de nacimiento
<class 'tuple'>
Error al extrear el año de nacimiento
<class 'tuple'>
Error al extrear el año de nacimiento
<class 'tuple'>
Error al extrear el año de nacimiento
<class 'tuple'>
Error al extrear el año de nacimiento
Error al encontrar las peliculas
Error al encontrar las peliculas
<class 'tuple'>
Error al extrear el año de nacimiento
Error al encontrar las peliculas
<class 'tuple'>
Error al extrear el año de nacimiento
Error al encontrar las peliculas
Error al encontrar las peliculas
<class 'tuple'>
Error al extrear el año de nacimiento
<class 'tuple'>
Error al extrear el año de nacimiento
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
Error al extrear el año de nacimiento
Error al extrear los premios
<class 'tuple'>
Error al extrear el año de nacimiento
Error al extrear los premios
<class 'tuple'>
Error al extrear el año de nacimiento
Error al extrear los premios
<class 'tuple'>
Error al extrear el año de nacimient

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (454, 581)
  (Session info: chrome=125.0.6422.60)
Stacktrace:
	GetHandleVerifier [0x00007FF7A4D622C2+60002]
	(No symbol) [0x00007FF7A4CDCA59]
	(No symbol) [0x00007FF7A4B97EDA]
	(No symbol) [0x00007FF7A4BEF39E]
	(No symbol) [0x00007FF7A4BECD62]
	(No symbol) [0x00007FF7A4BEA1FB]
	(No symbol) [0x00007FF7A4BE93C6]
	(No symbol) [0x00007FF7A4BDB4D1]
	(No symbol) [0x00007FF7A4C0C28A]
	(No symbol) [0x00007FF7A4BDADF6]
	(No symbol) [0x00007FF7A4C0C4A0]
	(No symbol) [0x00007FF7A4C2BCC7]
	(No symbol) [0x00007FF7A4C0C033]
	(No symbol) [0x00007FF7A4BD9657]
	(No symbol) [0x00007FF7A4BDA251]
	GetHandleVerifier [0x00007FF7A5073E2D+3278285]
	GetHandleVerifier [0x00007FF7A50C0190+3590448]
	GetHandleVerifier [0x00007FF7A50B61D0+3549552]
	GetHandleVerifier [0x00007FF7A4E11DE6+779654]
	(No symbol) [0x00007FF7A4CE7ACF]
	(No symbol) [0x00007FF7A4CE2EE4]
	(No symbol) [0x00007FF7A4CE3072]
	(No symbol) [0x000

In [ ]:
import pandas as pd

csv_url = "https://raw.githubusercontent.com/fernandaMarti/Proyecto-da-promo-H-modulo-2-team1-cine_freaks/Rama_Sharon_01/Fase3_concat_actores.csv"

df = pd.read_csv(csv_url)
df.columns=['0']
lista_detalles_actores=[]
lista_intermedia_actores = list(df['0'])
lista_intermedia_actores_2=lista_intermedia_actores[2450:2721]

In [ ]:
len(lista_detalles_actores)

In [7]:
df_lista_intermedia_actores_2 = pd.DataFrame(lista_detalles_actores)
df_lista_intermedia_actores_2.columns = ['nombre_actor','anno_nacimiento','peliculas','que_hace','premios']

In [8]:
df_lista_intermedia_actores_2

,nombre_actor,anno_nacimiento,peliculas,que_hace,premios
0,Macy Minear,0,"[All of You Will Die, Deadly Cheers, Share?, H...",[Reparto],1
1,Yana Sheuko,0,"[Hurtcore, The Host App, LIVE FEED: Breakdown,...",[Reparto],1
2,Sasha Prince,0,"[Hurtcore, Justice League Dark: Best of the Wo...","[Reparto, Decoración]",1
3,Spencer Vaughn Kelly,0,"[WordLotto, Hurtcore, The Shell, Bonus Trip]","[Reparto, Dirección, Guion]",4
4,Kaleb Anthony,0,"[Hurtcore, Still Standing, None, None]","[Reparto, Equipos adicionales]",1
...,...,...,...,...,...
121,Kris Alexandrea,0,"[Rot, Venganza personal, Gothic Springs, Unicorn]","[Reparto, Guion, Producción]",0
122,Kris Ann Russell,0,"[Saved by the Bell, Finding Love in Big Sky, M...","[Reparto, Guion, Dirección]",0
123,Linda Burzynski,0,"[Big Little Lies, Shameless, Animal Kingdom, F...","[Reparto, Producción, Guion]",3
124,Eileen Dietz,1944,"[Abnormal Attraction, David Holzman's Diary, H...","[Reparto, Producción]",2


In [9]:
df_lista_intermedia_actores_2.to_csv("lista_actores_2_parte2.csv", index = False)

In [33]:
print(len(lista_actores))
df = pd.DataFrame(lista_detalles_actores)
df

990


,0,1,2,3,4
0,Byom,0,"[Mom's Coming, Lage Na Mon, Love Driver, Jai H...","[Reparto, Producción, Dirección de la segunda ...",0
1,Rose LaRue,0,"[NULL, NULL, NULL, NULL]",NULL,0
2,Aaron Jackson,0,"[Rabos: El musical, The Chris Gethard Show: Pu...","[Reparto, Guion, Producción]",0
3,Becky Brown,0,"[Hanging Millstone, That's What Friends are Fo...","[Reparto, Producción, Casting]",0
4,Cassidy Reyes,0,"[The Bermuda Triangle Project, God Forbid, The...","[Reparto, Producción]",0
5,Mercedes Gutierrez,0,"[Beauty Is Skin Deep, Todd, Crescent City Chro...",[Reparto],0
6,Tay Lindsey,0,"[Step Up Revolution, Time 2 Tell, Quarry, Últi...","[Reparto, Equipos adicionales]",0
7,Deborah Smith Ford,0,[NULL],[NULL],0
8,Jason Menz,0,"[NULL, NULL, NULL, NULL]",NULL,0
9,Miles Christian-Hart,0,"[The 4th Letter of Ponce De Leon, Love Song an...",[Reparto],0
